In [ ]:
import os
import datetime
import cv2


############## user define #############
ImageSize = 32
DatasetType = ['mnist', 'cifar-10', 'linetrace', 'mnist-separate']
DatasetIndex = 2
epoch = 10
netName = 'VGG9ave'
binary = True
marker_type = 'cross'
img_num = 30000
separate = True
#####################################

DatasetName = DatasetType[DatasetIndex] + '_' + str(ImageSize)
ProjectName = 'Project_' + DatasetName + '_' + str(datetime.date.today()) + '_' + netName+ '_epoch' +str(epoch)
ProjectName = ProjectName + ('_imgnum' + str(img_num) + '_marker' + marker_type) if DatasetIndex == 2 else ''

def process(file_path):
    im = cv2.imread(file_path,0)
    ret, im = cv2.threshold(im,127,255,cv2.THRESH_BINARY_INV)
    cv2.imwrite(file_path, im)

def recursive_file_check(path):
    if os.path.isdir(path):
        files = os.listdir(path)
        for file in files:
            recursive_file_check(path + "/" + file)
    else:
        process(path)

!pip3 install 'cupy-cuda101>=7.7.0,<8.0.0'
!pip3 install chainer

%cd /content
if (os.path.exists('/content/CUINNESS')==False):
    !git clone https://github.com/knmrtkt/CUINNESS.git
else:
    !git pull
%cd CUINNESS
!git pull

%cd /content
if (DatasetType[DatasetIndex] == 'mnist'):
    if (os.path.exists('/content/mnist_png')==False):
        !git clone https://github.com/myleott/mnist_png.git
        %cd mnist_png
        !tar -zxvf mnist_png.tar.gz
        !rsync -a mnist_png/testing/ mnist_png/training
        if (binary == True):
            recursive_file_check('mnist_png/training/')
        !ls -1 -d ../mnist_png/mnist_png/training/* > list_tmp1.txt
        !ls -1 ../mnist_png/mnist_png/training > list_tmp2.txt
        !paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/list.txt
        !rm list_tmp1.txt list_tmp2.txt

elif (DatasetType[DatasetIndex] == 'cifar-10'):
    if (os.path.exists('/content/cifar2png')==False):
        !git clone https://github.com/knjcode/cifar2png.git
        %cd cifar2png
        !python3 cifar2png cifar10 cifar10png --name-with-batch-index
        !rsync -a cifar10png/test/ cifar10png/train
        %cd /content/CUINNESS
        !ls -1 -d ../cifar2png/cifar10png/train/* > list_tmp1.txt
        !ls -1 ../cifar2png/cifar10png/train > list_tmp2.txt
        !paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/list.txt
        !rm list_tmp1.txt list_tmp2.txt

elif (DatasetType[DatasetIndex] == 'linetrace'):
    if (os.path.exists('/content/linetrace')==False):
        
        from CUINNESS import gen_line_image
        img_gen = gen_line_image.gen_line_image()
        #img_gen.generate_dataset('linetrace/', img_num=img_num, class_label={"straight" : "0", "right_straight" : "1","left_straight" : "2",})
        img_gen.generate_image(ImageSize, ImageSize, 3, 2, 'linetrace/', 'dataset.csv', img_num, 4, {"straight" : "0", "right" : "1", "left" : "2",})
        !rm linetrace/dataset.csv
        img_gen.mark_image(ImageSize, ImageSize, 'linetrace', 4, 0, marker_type=marker_type)
        %cd /content/CUINNESS
        !ls -1 -d ../linetrace/* > list_tmp1.txt
        !ls -1 ../linetrace > list_tmp2.txt
        !paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/list.txt
        !rm list_tmp1.txt list_tmp2.txt

elif (DatasetType[DatasetIndex] == 'mnist-separate'):
    if (os.path.exists('/content/mnist_png')==False):
        !git clone https://github.com/myleott/mnist_png.git
        %cd mnist_png
        !tar -zxvf mnist_png.tar.gz
        if (binary == True):
            recursive_file_check('mnist_png/training/')
            recursive_file_check('mnist_png/testing/')
        !ls -1 -d ../mnist_png/mnist_png/training/* > list_tmp1.txt
        !ls -1 ../mnist_png/mnist_png/training > list_tmp2.txt
        !paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/trainlist.txt
        !rm list_tmp1.txt list_tmp2.txt
        !ls -1 -d ../mnist_png/mnist_png/testing/* > list_tmp1.txt
        !ls -1 ../mnist_png/mnist_png/testing > list_tmp2.txt
        !paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/testlist.txt
        !rm list_tmp1.txt list_tmp2.txt

%cd /content/CUINNESS
if (os.path.exists(DatasetName + '_dataset.pkl')==False):
    if(separate==True):
        !python3 gen_separate_data.py --trainpathfile trainlist.txt --testpathfile testlist.txt --dataset $DatasetName --size $ImageSize --keepaspect yes
    else:
        !python3 gen_training_data.py --pathfile list.txt --dataset $DatasetName --size $ImageSize --keepaspect yes
!python3 guinness.py $ProjectName $DatasetName $epoch --netName=$netName
%cd $ProjectName
!python3 eval.py --dataset ../$DatasetName --size $ImageSize --testnum 100 > eval_result.txt
%cd ..
ProjectZipPath = '/content/CUINNESS/' + ProjectName + '.zip'
!zip -r $ProjectZipPath /content/CUINNESS/$ProjectName
from google.colab import files
files.download(ProjectZipPath)